In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy

import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer, BertModel
from transformers.modeling_outputs import SequenceClassifierOutput
import numpy as np


df = pd.read_csv('poemsDataset.csv')
df.fillna('', inplace=True)
display(len(df))


def remove_diacritics(a):    
    return araby.strip_diacritics(a)

df['first_hemistich'] = df['first_hemistich'].apply(remove_diacritics)
df['second_hemistich'] = df['second_hemistich'].apply(remove_diacritics)

def normalizeBeforeTraining(df):
    df['first_hemistich'] = df['first_hemistich'].str.replace('النابغـة: ', '')
    df['second_hemistich'] = df['second_hemistich'].str.replace('الـربيع: ', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('عبيــد: ', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('امـرؤ القيسـ: ', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('امرؤ القيس: ', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('(جلال الــــدين الــــرومي):', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('(لـوك الفيلسـوف الإنكليزي):', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('(كانت الفيلسوف الألماني ):', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('(بركســــــــــــــــون):', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('(الحـــــــــــــــور):', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('(الشــــــــــــــاعر):', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('(الإنســـــــــــــــان):', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('العلم):', '', regex=False)
    df['first_hemistich'] = df['first_hemistich'].str.replace('(العشــــــــــــــــق):', '', regex=False)
    df['first_hemistich'] = df['first_hemistich'].str.replace('(الزهــــــــــــــــــرة):', '', regex=False)
    df['second_hemistich'] = df['second_hemistich'].str.replace('التوأم اليشكري: ', '', regex=False)  
    df['first_hemistich'] = df['first_hemistich'].str.replace('آ', 'أ')
    df['second_hemistich'] = df['second_hemistich'].str.replace('آ', 'أ')
    df['first_hemistich'] = df['first_hemistich'].str.replace('[/":?،؟]', '')
    df['second_hemistich'] = df['second_hemistich'].str.replace('[/":?،؟]', '')
    df['first_hemistich'] = df['first_hemistich'].str.replace('  ', ' ')
    df['second_hemistich'] = df['second_hemistich'].str.replace('  ', ' ')
    df['first_hemistich'] = df['first_hemistich'].str.replace('  ', ' ')
    df['second_hemistich'] = df['second_hemistich'].str.replace('  ', ' ')


normalizeBeforeTraining(df)
df.drop(df[(df['first_hemistich'] == '') & (df['second_hemistich'] == '')].index, inplace=True)

#if first_hemistich == '', then copy the text from second_hemistich. then delete the text in the second_hemistich
df['first_hemistich'] = df.apply(lambda x: x['second_hemistich'] if x['first_hemistich'] == '' else x['first_hemistich'], axis=1)
df['second_hemistich'] = df.apply(lambda x: '' if x['first_hemistich'] == x['second_hemistich'] else x['second_hemistich'], axis=1)

df.reset_index(drop=True, inplace=True)

display(len(df))
# display(df[:10])
print('done')


df['second_hemistich'].replace('', 'E', inplace=True)
dfc = df[['first_hemistich', 'second_hemistich', 'meter', 'link']].copy()
dfc['text'] = dfc['first_hemistich'] + ' S ' + dfc['second_hemistich']

#removing verses without a meter
dfc = dfc[dfc['meter'] != ''] 
dfc = dfc[dfc['meter'] != 'unspecified']
dfc = dfc[dfc['meter'] != 'mixed']



classic = ['taweel', 'kamel', 'baseet', 'khafif', 'wafer', 'rajaz', 'ramel', 'mutaqarib',
           'saree', 'munsarih', 'mujtath', 'hazaj', 'madeed', 'mutadarak', 'muqtadab', 'mudari'] 

# including only verses with non-classical meters
dfc = dfc[~dfc['meter'].isin(classic)]

dfc.reset_index(drop=True, inplace=True)

dfc['meter'] = dfc['meter'].astype('category')
# display(dfc['meter'].unique())

dfc['label'] = dfc['meter'].cat.codes 


df = dfc[['text', 'label']]

classes_num = len(dfc['label'].unique())
# classes_num = len(classes)
display(classes_num)
display(len(df))



ds = Dataset.from_pandas(df)

ds = ds.train_test_split(test_size=0.2)
display(ds)

2024-10-08 06:35:26.942741: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-08 06:35:26.969083: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-08 06:35:27.432919: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2090907

2090907

done


12

61502

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 49201
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12301
    })
})

In [3]:
log_file = 'non_classical.txt'
with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')


max_sequence_length = 32
train_batch_size = 256

models = [ 
        'aubmindlab/bert-base-arabertv01',
    'aubmindlab/bert-base-arabert',
        'CAMeL-Lab/bert-base-arabic-camelbert-ca',
        'qarib/bert-base-qarib', 
        'UBC-NLP/ARBERT',
        'faisalq/bert-base-arapoembert'
]

for model_name in models:
    for i in range(3):
        # ds = Dataset.from_pandas(dfx)
        # ds = ds.train_test_split(test_size=0.2, seed=i)
        
        print(f'{model_name}, try:{i}')
              
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=classes_num).to('cuda')                                                 
        
        dataset_train = ds['train']
        dataset_validation = ds['test']                                                    
        
        def preprocess_function(examples):
            return tokenizer(examples['text'], truncation=True, padding="max_length",
                             max_length=max_sequence_length)
        
        dataset_train = dataset_train.map(preprocess_function, batched=True)
        dataset_validation = dataset_validation.map(preprocess_function, batched=True)
        
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)    
            acc = accuracy_score(labels, predictions)        
            f1 = f1_score(labels, predictions, average='macro')
            with open(log_file, 'a') as f:
                f.write(f'{model_name},{acc},{f1}\n')
            return {'accuracy': acc, 'f1_score': f1}
        
        # Training setup
        epochs = 8
        save_steps = 10000
        batch_size = 64
        
        training_args = TrainingArguments(
            output_dir='bert/',
            overwrite_output_dir=True,
            num_train_epochs=epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            save_steps=save_steps,
            save_total_limit=1,
            fp16=True,
            learning_rate=5e-5,
            logging_steps=200,
            evaluation_strategy='steps',
            eval_steps=200
        )
        
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=dataset_train,
            eval_dataset=dataset_validation,
            compute_metrics=compute_metrics
        )
        
        trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'Accuracy'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv('non_classical.csv')
display(best_results)


aubmindlab/bert-base-arabertv01, try:0


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv01 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/49201 [00:00<?, ? examples/s]

Map:   0%|          | 0/12301 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
200,0.840200,0.719320,0.781237,0.225388
400,0.665300,0.641442,0.791562,0.266050
600,0.619700,0.588865,0.810341,0.347279
800,0.550300,0.536937,0.834079,0.363527
1000,0.422400,0.572427,0.831884,0.372549
1200,0.414700,0.506284,0.845297,0.392271
1400,0.407600,0.496134,0.841558,0.388213
1600,0.357700,0.537662,0.845622,0.384566
1800,0.243300,0.558245,0.848386,0.400998
2000,0.249800,0.537505,0.850012,0.395597


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at faisalq/bert-base-arapoembert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/49201 [00:00<?, ? examples/s]

Map:   0%|          | 0/12301 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
200,0.558100,0.404447,0.882367,0.447939
400,0.378900,0.374092,0.889521,0.461793
600,0.344900,0.336972,0.893992,0.473000
800,0.310100,0.355340,0.900984,0.480249
1000,0.193600,0.378750,0.901065,0.482947
1200,0.195600,0.330107,0.907243,0.492358
1400,0.184500,0.321083,0.906186,0.487712
1600,0.157800,0.396964,0.906999,0.491375
1800,0.083000,0.419521,0.908381,0.492268
2000,0.081700,0.433245,0.909032,0.499760


KeyError: 'Accuracy'

In [5]:
results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['Accuracy'].max()

best_results = pd.merge(best_results, results, on=['Model', 'Accuracy'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv('non_classical.csv')
display(best_results)

,Model,Accuracy,F1
0,CAMeL-Lab/bert-base-arabic-camelbert-ca,0.890009,0.555142
3,UBC-NLP/ARBERT,0.856191,0.427453
6,aubmindlab/bert-base-arabert,0.849524,0.412987
9,aubmindlab/bert-base-arabertv01,0.860418,0.472605
10,faisalq/bert-base-arapoembert,0.912284,0.555195
13,qarib/bert-base-qarib,0.884887,0.559053
